## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [144]:
 # Asegúrate de que pandas esté importado

class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self._precio = precio if precio >= 0 else 0  # Validación inicial
        self.tipo = tipo
        self._cantidad = cantidad if cantidad >= 0 else 0  # Validación inicial

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, value):
        if value >= 0:  # Validación para el precio
            self._precio = value

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, value):
        if value >= 0:  # Validación para la cantidad
            self._cantidad = value

    def valorTotal(self):
        """Calcula el valor total del producto en base a su cantidad en existencia"""
        return self.precio * self.cantidad

    def __str__(self):
        """Devuelve una representación en cadena del producto"""
        return f"Producto({self.codigo}, {self.nombre}, Precio: {self.precio}, Tipo: {self.tipo}, Cantidad: {self.cantidad})"


In [145]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [93]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.precio = precio
        self.tipo = tipo
        self.cantidad = cantidad


class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos  # Lista de códigos de productos a los que aplica la oferta
        self.tipos = tipos      # Lista de tipos de productos a los que aplica la oferta

    def es_applicable(self, producto):
        # La oferta se aplica si el producto coincide con algún código o tipo
        return producto.codigo in self.codigos or producto.tipo in self.tipos


class OfertaDescuento(Oferta):
    def __init__(self, descuento, descripcion="Descuento", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)
        self.descuento = descuento  # El porcentaje de descuento

    def calcularDescuento(self, producto, cantidad):
        if self.es_applicable(producto):
            descuento_total = producto.precio * self.descuento / 100 * cantidad
            return descuento_total
        return 0


class Oferta2x1(Oferta):
    def __init__(self, descripcion="2x1", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        if self.es_applicable(producto) and cantidad >= 2:
            # Calcula el descuento por cada par de productos (2x1)
            pares = cantidad // 2
            descuento_total = pares * producto.precio
            return descuento_total
        return 0




In [83]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [138]:
import csv

class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.precio = precio
        self.tipo = tipo
        self.cantidad = cantidad

class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos

    def aplicable(self, producto, cantidad):
        # Verifica si el producto es aplicable a la oferta
        return producto.codigo in self.codigos or producto.tipo in self.tipos

class OfertaDescuento(Oferta):
    def __init__(self, descuento, codigos=[], tipos=[]):
        super().__init__(descripcion=f"Descuento del {descuento}%", codigos=codigos, tipos=tipos)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if producto.codigo in self.codigos:
            return producto.precio * cantidad * (self.descuento / 100)
        return 0

class Oferta2x1(Oferta):
    def __init__(self, tipos=[], codigos=[]):
        super().__init__(descripcion="Oferta 2x1", tipos=tipos, codigos=codigos)  # Llama al constructor de Oferta

    def calcularDescuento(self, producto, cantidad):
        if self.aplicable(producto, cantidad):  # Usa el método aplicable para verificar
            return (cantidad // 2) * producto.precio  # Calcula el descuento por cada par
        return 0

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        return sum(producto.precio * producto.cantidad for producto in self.productos)

    def agregar(self, producto):
        self.productos.append(producto)

    def calcularDescuento(self, producto, cantidad):
        mejor_descuento = 0
        for oferta in self.ofertas:
            descuento = oferta.calcularDescuento(producto, cantidad)
            if descuento > mejor_descuento:
                mejor_descuento = descuento
        return mejor_descuento

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def vender(self, producto, cantidad):
        for p in self.productos:
            if p.codigo == producto.codigo:
                if p.cantidad >= cantidad:
                    p.cantidad -= cantidad
                break

    def guardar(self, archivo):
        with open(archivo, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        try:
            with open(archivo, 'r') as f:
                print(f"Leyendo archivo {archivo}...")
                reader = csv.DictReader(f)
                for row in reader:
                    print(f"Leyendo fila: {row}")  # Imprimir cada fila leída
                    p = Producto(row['codigo'], row['nombre'], float(row['precio']), row['tipo'], int(row['cantidad']))
                    catalogo.agregar(p)
                print("Lectura del archivo finalizada.")
        except FileNotFoundError:
            print(f"Error: el archivo {archivo} no se encuentra.")
            return None
        except Exception as e:
            print(f"Error durante la lectura del archivo: {e}")
            return None

        # Bloque para imprimir el contenido del catálogo después de la lectura
        print("Contenido del catálogo después de leer:")
        for producto in catalogo.productos:
            print(f"{producto.codigo}, {producto.nombre}, {producto.precio}, {producto.tipo}, {producto.cantidad}")
            
        return catalogo

    def informe(self):
        informe = f"INFORME CATALOGO\nCantidad de productos: {self.cantidadProductos}\n"
        informe += f"Cantidad de unidades: {self.cantidadUnidades}\n"
        
        # Calcular el precio promedio
        precio_promedio = self.valorTotal / self.cantidadUnidades if self.cantidadUnidades > 0 else 0
        informe += f"Precio Promedio: $ {precio_promedio:.2f}\n"
        informe += f"Valor total: $ {self.valorTotal:.2f}\n"
        
        # Asegurarse de que se agregue correctamente "Tipos de productos:"
        informe += "Tipos de productos: \n"  # Aseguramos que el formato sea exacto

        # Contar los tipos de productos
        tipos = {}
        for producto in self.productos:
            if producto.tipo not in tipos:
                tipos[producto.tipo] = {"unidades": 0, "total_precio": 0}
            tipos[producto.tipo]["unidades"] += producto.cantidad
            tipos[producto.tipo]["total_precio"] += producto.precio * producto.cantidad

        for tipo, datos in tipos.items():
            unidades = datos["unidades"]
            promedio_precio = datos["total_precio"] / unidades if unidades > 0 else 0
            informe += f"  - {tipo} : {unidades}u x $ {promedio_precio:.2f}\n"

        informe += "Ofertas:\n"
        for oferta in self.ofertas:
            informe += f"  - {oferta.descripcion}\n"
        
        return informe

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

In [104]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Leyendo archivo catalogo-prueba.csv...
Leyendo fila: {'codigo': '0001', 'nombre': 'Coca Cola', 'precio': '1500', 'tipo': 'gaseosa', 'cantidad': '10'}
Leyendo fila: {'codigo': '0002', 'nombre': 'Pepsi Cola', 'precio': '1200', 'tipo': 'gaseosa', 'cantidad': '20'}
Leyendo fila: {'codigo': '0003', 'nombre': 'Sonrisa', 'precio': '1200', 'tipo': 'galleta', 'cantidad': '30'}
Leyendo fila: {'codigo': '0004', 'nombre': 'Oreo', 'precio': '2300', 'tipo': 'galleta', 'cantidad': '40'}
Lectura del archivo finalizada.
Contenido del catálogo después de leer:
0001, Coca Cola, 1500.0, gaseosa, 10
0002, Pepsi Cola, 1200.0, gaseosa, 20
0003, Sonrisa, 1200.0, galleta, 30
0004, Oreo, 2300.0, galleta, 40
Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [86]:
import re

class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = None
        self._cuit = None
        self.nombre = nombre  # Usamos el setter para validar
        self.cuit = cuit      # Usamos el setter para validar

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        # Validar que el nombre tenga entre 1 y 100 caracteres y no sea vacío
        if 1 <= len(valor) <= 100:
            self._nombre = valor
        else:
            pass  # Rechaza la modificación si no cumple con las condiciones

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, valor):
        # Validar el CUIT con el formato XX-XXXXXXXX-X
        if re.fullmatch(r'\d{2}-\d{8}-\d', valor):
            self._cuit = valor
        else:
            pass  # Rechaza la modificación si no cumple con las condiciones

    def __repr__(self):
        return f"Cliente(nombre='{self.nombre}', cuit='{self.cuit}')"



In [87]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [142]:
from datetime import datetime

# Definición de todas las clases necesarias

class Producto:
    def __init__(self, codigo, nombre, precio, tipo, stock):
        self.codigo = codigo
        self.nombre = nombre
        self.precio = precio
        self.tipo = tipo
        self.stock = stock


class Catalogo:
    def __init__(self):
        self.productos = {}  # Diccionario para almacenar productos
        self.ofertas = []     # Lista para almacenar ofertas

    def agregar(self, *productos):
        for producto in productos:
            self.productos[producto.codigo] = producto  # Agrega el producto al diccionario usando su código como clave

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)  # Agrega una oferta a la lista de ofertas

    def buscar(self, codigo):
        return self.productos.get(codigo, None)  # Busca un producto por su código y devuelve None si no se encuentra


class Cliente:
    def __init__(self, nombre, cuit):
        self.nombre = nombre
        self.cuit = cuit




#class Oferta:
    #def aplicable(self, producto, cantidad):
        #raise NotImplementedError("Debe implementarse en una subclase")
    
    #def calcularDescuento(self, producto, cantidad):
        #raise NotImplementedError("Debe implementarse en una subclase")
    
    
    #def descripcion(self):
        #raise NotImplementedError("Debe implementarse en una subclase")
#class Oferta2x1(Oferta):
    #def __init__(self, tipos):
        #self.tipos = tipos
    
    #def aplicable(self, producto, cantidad):
       # return producto.tipo in self.tipos and cantidad >= 2
    
    
    #def calcularDescuento(self, producto, cantidad):
        #if self.aplicable(producto, cantidad):
            #return (cantidad // 2) * producto.precio
        #return 0
    
    #def descripcion(self):
        #return "Oferta 2x1"
#class OfertaDescuento(Oferta):
    #def __init__(self, porcentaje, codigos):
        #self.porcentaje = porcentaje
        #self.codigos = codigos
    
    #def aplicable(self, producto, cantidad):
        #return producto.codigo in self.codigos
    
   # def calcularDescuento(self, producto, cantidad):
       # if self.aplicable(producto, cantidad):
       #     return (producto.precio * cantidad) * (self.porcentaje / 100)
       # return 0
    
    #def descripcion(self):
       # return f"Descuento {self.porcentaje}%"





class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos

    def aplicable(self, producto, cantidad):
        # Verifica si el producto es aplicable a la oferta
        return producto.codigo in self.codigos or producto.tipo in self.tipos


class OfertaDescuento(Oferta):
    def __init__(self, descuento, codigos=[], tipos=[]):
        super().__init__(descripcion=f"Descuento del {descuento}%", codigos=codigos, tipos=tipos)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if producto.codigo in self.codigos:
            return producto.precio * cantidad * (self.descuento / 100)
        return 0

class Oferta2x1(Oferta):
    def __init__(self, tipos=[], codigos=[]):
        super().__init__(descripcion="Oferta 2x1", tipos=tipos, codigos=codigos)  # Llama al constructor de Oferta

    def calcularDescuento(self, producto, cantidad):
        if self.aplicable(producto, cantidad):  # Usa el método aplicable para verificar
            return (cantidad // 2) * producto.precio  # Calcula el descuento por cada par
        return 0
class Factura:
    numero_actual = 100
    
    def __init__(self, catalogo, cliente):
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.now().strftime("%Y-%m-%d")  # Fecha actual
        self.cliente = cliente
        self.catalogo = catalogo
        self.productos = {}  # Diccionario para almacenar productos agregados
        self.subtotal = 0
        self.descuentos = 0
        self.total = 0
    
    @classmethod
    def ultimaFactura(cls, numero):
        cls.numero_actual = numero
    
    @classmethod
    def nuevoNumero(cls):
        cls.numero_actual += 1
        return cls.numero_actual
    
    def agregar(self, producto, cantidad):
        if producto is None:
            print("Producto no encontrado")
            return

        # Agregar lógica para agregar el producto
        if producto.codigo in self.productos:
            self.productos[producto.codigo]["cantidad"] += cantidad
        else:
            self.productos[producto.codigo] = {
                "producto": producto,
                "cantidad": cantidad,
                "descuento": 0,
                "descripcion_oferta": ""
            }
        self.calcularTotales()  # Asegúrate de que esta llamada sea correcta
    
    def calcularTotales(self):
        self.subtotal = 0
        self.descuentos = 0
        
        for item in self.productos.values():
            producto = item["producto"]
            cantidad = item["cantidad"]
            subtotal_producto = producto.precio * cantidad
            self.subtotal += subtotal_producto
            
            # Buscar la mejor oferta aplicable
            mejor_descuento = 0
            descripcion_mejor_oferta = ""
            for oferta in self.catalogo.ofertas:
                if oferta.aplicable(producto, cantidad):
                    descuento = oferta.calcularDescuento(producto, cantidad)
                    if descuento > mejor_descuento:
                        mejor_descuento = descuento
                        descripcion_mejor_oferta = oferta.descripcion()
            
            # Aplicar solo la mejor oferta
            item["descuento"] = mejor_descuento
            item["descripcion_oferta"] = descripcion_mejor_oferta
            self.descuentos += mejor_descuento
        
        self.total = self.subtotal - self.descuentos
    
    @property
    def cantidadProductos(self):
        return len(self.productos)
    
    @property
    def cantidadUnidades(self):
        return sum(item["cantidad"] for item in self.productos.values())
    
    def imprimir(self):
        factura_texto = f"Factura: {self.numero}\n"
        factura_texto += f"Fecha  : {self.fecha}\n"
        factura_texto += f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n\n"
        
        for item in self.productos.values():
            producto = item["producto"]
            cantidad = item["cantidad"]
            subtotal_producto = producto.precio * cantidad
            factura_texto += f"- {cantidad} {producto.nombre:<30} x ${producto.precio:.0f} = ${subtotal_producto:.0f}\n"
            
            if item["descuento"] > 0:
                factura_texto += f"      {item['descripcion_oferta']:<40} - ${item['descuento']:.0f}\n"
        
        factura_texto += f"\n{'':>40}Subtotal:   ${self.subtotal:.0f}\n"
        factura_texto += f"{'':>40}Descuentos: ${self.descuentos:.1f}\n"
        factura_texto += f"{'':>40}-----------------------\n"
        factura_texto += f"{'':>40}TOTAL:      ${self.total:.2f}\n"  # Cambiado a .2f para dos decimales
        
        return factura_texto


In [143]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)
# Agrego los productos uno por uno


# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

TypeError: 'str' object is not callable

In [139]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Leyendo archivo catalogo.csv...
Leyendo fila: {'codigo': '0001', 'nombre': 'Coca Cola', 'precio': '1490', 'tipo': 'gaseosa', 'cantidad': '10'}
Leyendo fila: {'codigo': '0002', 'nombre': 'Pepsi Cola', 'precio': '1200', 'tipo': 'gaseosa', 'cantidad': '20'}
Leyendo fila: {'codigo': '0003', 'nombre': 'Sonrisa', 'precio': '1200', 'tipo': 'galleta', 'cantidad': '30'}
Leyendo fila: {'codigo': '0004', 'nombre': 'Oreo', 'precio': '2300', 'tipo': 'galleta', 'cantidad': '40'}
Leyendo fila: {'codigo': '0005', 'nombre': 'Fanta', 'precio': '1400', 'tipo': 'gaseosa', 'cantidad': '15'}
Leyendo fila: {'codigo': '0006', 'nombre': 'Sprite', 'precio': '1300', 'tipo': 'gaseosa', 'cantidad': '25'}
Leyendo fila: {'codigo': '0007', 'nombre': '7 Up', 'precio': '1350', 'tipo': 'gaseosa', 'cantidad': '18'}
Leyendo fila: {'codigo': '0008', 'nombre': 'Dr Pepper', 'precio': '1600', 'tipo': 'gaseosa', 'cantidad': '32'}
Leyendo fila: {'codigo': '0009', 'nombre': 'Chokis', 'precio': '2500', 'tipo': 'galleta', 'cantida

TypeError: 'str' object is not callable